In [14]:
from pymongo import MongoClient
from pprint import pprint
import json
from inventory import Inventory, reconstruct_inventory_states
import list_tricks

with open('all_items.json') as f:
    allthings = json.load(f)

allcards = set(allthings['cards'])
allrelics = set(allthings['relics'])

client = MongoClient('mongodb://localhost:27017')
db = client.spire
status = db.command('serverStatus')
runs = db.runs

search = {
    'event.character_chosen': 'IRONCLAD', 
    'event.floor_reached': {
        '$gte': 40
    }, 
    'event.ascension_level': {
        '$gte': 15
    },
    "event.relics": {
        '$nin': ['PrismaticShard']
    },
    "event.is_beta": False,
    "event.is_endless": False,
    # "event.relics": {"$in": ["Peace Pipe"]}, # todo: remove
    
}

num_runs = runs.count_documents(search)
print('num_runs', num_runs)

print("\nstarting reconstruction...")

result = runs.aggregate([
    {'$match': search},
    # {'$sample': { 'size': 10 } } # todo: remove
])

skipped = 0
used = 0
total = 0

for r in result:
    run = r['event']
    
    inventory_states = reconstruct_inventory_states(run, allcards, allrelics)

    md_sorted = sorted(run['master_deck'])
    deck_sorted = sorted(inventory_states[max(inventory_states.keys())]['inventory']['deck'])
    compare = list_tricks.compare_lists(md_sorted, deck_sorted)

    # if md_sorted == deck_sorted:
    #     print("DECKS EQUAL")
    # print('\nactual deck:\n', md_sorted)
    # print('\nmy reconstruction:\n', deck_sorted)
    # print('\nin original but not in mine:\n', compare['a-b'])
    # print('\nin mine but not in original:\n', compare['b-a'])
    # print('\ndiff ratio: ', compare['diff_ratio'])
    # print('\n\n')

    if compare['diff_ratio'] > .1 :
        skipped += 1
    else:
        used += 1
        db.snapshots.insert_many([snapshot for floor, snapshot in inventory_states.items()])
        
    total += 1
    
    if total % 10 == 0:
        print("processed ", total, " (used ", 100 * used / total, "%)")

print("used: ", used)
print("skipped : ", skipped)

num_runs 10003

starting reconstruction...
card not found (remove):  Parasite
processed  10  (used  80.0 %)
card not found (remove):  Shame
card not found (upgrade):  Limit Break
card not found (remove):  Inflame
card not found (remove):  Normality
processed  20  (used  75.0 %)
relic not found:  NeowsBlessing
card not found (remove):  Sword Boomerang
processed  30  (used  70.0 %)
processed  40  (used  75.0 %)
card not found (remove):  Strike_R
card not found (remove):  Regret
processed  50  (used  74.0 %)
card not found (remove):  Blood for Blood
card not found (remove):  Bludgeon+1
card not found (remove):  Clumsy
processed  60  (used  73.33333333333333 %)
card not found (remove):  Clumsy
card not found (remove):  Shame
card not found (remove):  Injury
card not found (remove):  Bloodletting
card not found (remove):  Ghostly Armor
card not found (remove):  Exhume
card not found (upgrade):  Shockwave
processed  70  (used  74.28571428571429 %)
processed  80  (used  73.75 %)
card not foun